# Environment Test Setup

## Imports

In [42]:
from kaggle_environments import evaluate, make, utils
import numpy as np

## Test Setup

In [ ]:
env = make("connectx", debug=True)
env.render()

In [ ]:
# 1. Create the environment
env = make("connectx", debug=True)

# 2. You must 'run' or 'reset' the environment for a board to exist
env.reset()

# 3. Explicitly print the human-readable render
print(env.render(mode="ansi"))

# Agent Definitions

## Random Agent

In [43]:
def random_agent(observation, configuration):
    """
    A simple agent that plays a random, but valid, move.
    """
    import random
    # observation.board is a 1D list representing the grid.
    # 0 = empty, 1 = player 1, 2 = player 2.
    # The top row corresponds to indices 0 through (configuration.columns - 1).
    
    # Find all columns that are not completely full
    valid_moves = [
        col for col in range(configuration.columns) 
        if observation.board[col] == 0
    ]
    
    # Return a random choice among the valid columns
    return random.choice(valid_moves)

## Leftmost Agent

In [44]:
def leftmost_agent(observation, configuration):
    """Always picks the leftmost valid column."""
    valid_moves = [col for col in range(configuration.columns) if observation.board[col] == 0]
    return valid_moves[0]

## Reactive Agent
1. Level 1: The One-Move Lookahead (Reactive)
The Goal: Build an agent that isn't blind. It should see a winning move and take it, and see an opponent’s winning move and block it.

Concepts: Simulating moves, identifying winning states, and basic heuristics.

Outcome: A "Smart-ish" agent that beats Random and Leftmost 100% of the time.

In [32]:
def drop_piece(grid, col, piece, config):
    """
    Simulates dropping a piece into a specific column of the board.

    Args:
        grid (np.ndarray): A 2D numpy array representing the current board state.
        col (int): The index of the column (0 to config.columns-1) where the piece is dropped.
        piece (int): The player's mark (1 or 2).
        config (struct): The game configuration containing board dimensions (rows, columns).

    Returns:
        np.ndarray: A new 2D numpy array representing the board AFTER the move.
                   Returns the original grid if the move is invalid (column full).
    """
    # Create a deep copy so we don't modify the original 'real' board
    next_grid = grid.copy()
    
    # We loop from the bottom row up to the top row
    # range(start, stop, step) -> (bottom_index, -1, go_up_by_one)
    for row in range(config.rows - 1, -1, -1):
        if next_grid[row][col] == 0:
            next_grid[row][col] = piece
            return next_grid
            
    # If we get here, the column was full
    return next_grid

In [35]:
def is_win(grid, piece, config):
    """
    Scans the board to determine if the specified player has won.

    Args:
        grid (np.ndarray): The 2D numpy array representing the board.
        piece (int): The player mark to check for (1 or 2).
        config (struct): Configuration containing 'rows', 'columns', and 'inarow'.

    Returns:
        bool: True if the player has 'config.inarow' in a line, False otherwise.
    """
    # 1. Check Horizontal
    for r in range(config.rows):
        for c in range(config.columns - (config.inarow - 1)):
            window = list(grid[r, c:c+config.inarow])
            if window.count(piece) == config.inarow:
                return True

    # 2. Check Vertical
    for c in range(config.columns):
        for r in range(config.rows - (config.inarow - 1)):
            window = list(grid[r:r+config.inarow, c])
            if window.count(piece) == config.inarow:
                return True

    # 3. Check Positive Diagonal (/)
    for r in range(config.rows - (config.inarow - 1)):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r+i, c+i] for i in range(config.inarow)]
            if window.count(piece) == config.inarow:
                return True

    # 4. Check Negative Diagonal (\)
    for r in range(config.inarow - 1, config.rows):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r-i, c+i] for i in range(config.inarow)]
            if window.count(piece) == config.inarow:
                return True

    return False

### Unit Tests

In [34]:
# Test drop_piece()
class MockConfig:
    def __init__(self, rows, columns):
        self.rows = rows
        self.columns = columns

def test_drop_piece_advanced():
    config_2x2 = MockConfig(rows=2, columns=2)
    board = np.zeros((2, 2), dtype=int)
    print("Step 0: Empty 2x2 Board\n", board)
    
    # 1) Drop piece 1 in column 0
    board = drop_piece(board, col=0, piece=1, config=config_2x2)
    print("\nStep 1: P1 in Col 0\n", board)
    assert board[1][0] == 1
    
    # 2) Drop piece 2 in column 1
    board = drop_piece(board, col=1, piece=2, config=config_2x2)
    print("\nStep 2: P2 in Col 1\n", board)
    assert board[1][1] == 2
    
    # 3) Drop piece 1 in column 0
    board = drop_piece(board, col=0, piece=1, config=config_2x2)
    print("\nStep 3: P1 in Col 0 (should stack)\n", board)
    assert board[0][0] == 1
    
    # 4) Drop piece 2 in column 0 (Wait, this column is already full!)
    # Since it's a 2x2 board, Column 0 now has two pieces. 
    # Let's see how our function handles a 3rd piece in a 2-high column.
    board_after_full = drop_piece(board, col=0, piece=2, config=config_2x2)
    print("\nStep 4: P2 in FULL Col 0 (should not change board)\n", board_after_full)
    
    # Verification
    assert np.array_equal(board, board_after_full), "Error: Board changed even though column was full!"
    print("\n✅ All tests passed! The simulator correctly handles stacking and full columns.")

test_drop_piece_advanced()

Step 0: Empty 2x2 Board
 [[0 0]
 [0 0]]

Step 1: P1 in Col 0
 [[0 0]
 [1 0]]

Step 2: P2 in Col 1
 [[0 0]
 [1 2]]

Step 3: P1 in Col 0 (should stack)
 [[1 0]
 [1 2]]

Step 4: P2 in FULL Col 0 (should not change board)
 [[1 0]
 [1 2]]

✅ All tests passed! The simulator correctly handles stacking and full columns.


In [36]:
# Test is_win()
class MockConfig:
    def __init__(self, rows, columns, inarow):
        self.rows = rows
        self.columns = columns
        self.inarow = inarow

def test_is_win():
    # Setup: 3x3 board where you only need 2-in-a-row to win
    config_3x3 = MockConfig(rows=3, columns=3, inarow=2)
    
    # Test 1: Horizontal Win
    grid_h = np.array([
        [0, 0, 0],
        [0, 0, 0],
        [1, 1, 0]  # Two 1s in the bottom row
    ])
    assert is_win(grid_h, 1, config_3x3) == True
    assert is_win(grid_h, 2, config_3x3) == False # Player 2 hasn't won
    print("Test 1: Horizontal Passed")

    # Test 2: Vertical Win
    grid_v = np.array([
        [2, 0, 0],
        [2, 0, 0],
        [1, 0, 0]
    ])
    assert is_win(grid_v, 2, config_3x3) == True
    print("Test 2: Vertical Passed")

    # Test 3: Positive Diagonal (/)
    grid_pd = np.array([
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 0]
    ])
    assert is_win(grid_pd, 1, config_3x3) == True
    print("Test 3: Positive Diagonal Passed")

    # Test 4: Negative Diagonal (\)
    grid_nd = np.array([
        [2, 0, 0],
        [0, 2, 0],
        [0, 0, 0]
    ])
    assert is_win(grid_nd, 2, config_3x3) == True
    print("Test 4: Negative Diagonal Passed")

    # Test 5: No Win
    grid_none = np.array([
        [1, 2, 1],
        [2, 1, 2],
        [1, 2, 1]
    ])
    # Even though it's full, nobody has 2-in-a-row? 
    # Wait, in a 3x3 with 2-in-a-row, almost any move wins.
    # Let's use an empty board.
    grid_empty = np.zeros((3,3))
    assert is_win(grid_empty, 1, config_3x3) == False
    print("Test 5: Empty Board Passed")

    print("\n✅ All Vision Tests Passed! The agent can now see.")

test_is_win()

Test 1: Horizontal Passed
Test 2: Vertical Passed
Test 3: Positive Diagonal Passed
Test 4: Negative Diagonal Passed
Test 5: Empty Board Passed

✅ All Vision Tests Passed! The agent can now see.


# Arena Play
Play a game between two agents

In [ ]:
from kaggle_environments import make
env = make("connectx", debug=True)
env.run([random_agent, "negamax"])
env.render(mode="ipython", width=500, height=450)

Play Your Agent

In [ ]:
# "None" represents which agent you'll manually play as (first or second player).
from kaggle_environments import make
env = make("connectx", debug=True)
env.play([None, "negamax"], width=500, height=450)

# Evaluation
Evaluate the agent in batch test runs against benchmark agents

In [25]:
def agent_battle(agent1, agent2, n_rounds=100):
    # Get names for printing
    name1 = agent1.__name__ if hasattr(agent1, '__name__') else str(agent1)
    name2 = agent2.__name__ if hasattr(agent2, '__name__') else str(agent2)
    
    config = {'rows': 6, 'columns': 7, 'inarow': 4}

    # Run the rounds
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds // 2)
    outcomes += [[b, a] for [a, b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds - n_rounds // 2)]
    
    # Calculate stats based on corrected rewards
    # outcomes.count([1, 0]) = Agent 1 wins
    # outcomes.count([0.5, 0.5]) = Ties
    # outcomes.count([None, 1]) = Agent 1 crashed
    
    a1_wins = outcomes.count([1, 0])
    a2_wins = outcomes.count([0, 1])
    ties = outcomes.count([0.5, 0.5])
    invalid = len(outcomes) - (a1_wins + a2_wins + ties)
    
    print(f"--- Results: {name1} vs {name2} ---")
    print(f"{name1} Win Rate: {np.round(a1_wins / len(outcomes), 2)}")
    print(f"{name2} Win Rate: {np.round(a2_wins / len(outcomes), 2)}")
    print(f"Ties: {np.round(ties / len(outcomes), 2)}")
    print(f"Invalid Plays (Crashes): {np.round(invalid / len(outcomes), 2)}")

## Random vs Negamax

In [16]:
n_rounds = 10
print("Random vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle(random_agent, "negamax", n_rounds)

Random vs Negamax
2 rounds...
[[0, 1], [0, 1]]
--- Results: random_agent vs negamax ---
random_agent Win Rate: 0.0
negamax Win Rate: 1.0
Ties: 0.0
Invalid Plays (Crashes): 0.0


## Leftmost vs Random

In [26]:
n_rounds = 100
print("Random vs Leftmost")
print(str(n_rounds) + " rounds...")
agent_battle(random_agent, leftmost_agent, n_rounds)

Random vs Leftmost
100 rounds...
--- Results: random_agent vs leftmost_agent ---
random_agent Win Rate: 0.15
leftmost_agent Win Rate: 0.85
Ties: 0.0
Invalid Plays (Crashes): 0.0


## Negamax vs Negamax

In [29]:
n_rounds = 100
print("Negamax vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle("negamax", "negamax", n_rounds)

Negamax vs Negamax
100 rounds...
--- Results: negamax vs negamax ---
negamax Win Rate: 0.5
negamax Win Rate: 0.5
Ties: 0.0
Invalid Plays (Crashes): 0.0


# Custom 2D Board Rendering

In [ ]:
import numpy as np

def render_board(observation, configuration):
    # Convert the flat list to a 2D numpy array (Matrix)
    # This is exactly how your AI will 'see' the board
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    
    print("\n  0 1 2 3 4 5 6") # Column headers
    print(" ---------------")
    for row in grid:
        # Replace numbers with clearer symbols: . (empty), X (P1), O (P2)
        row_str = " ".join(['.' if x == 0 else ('X' if x == 1 else 'O') for x in row])
        print(f"| {row_str} |")
    print(" ---------------")

# Test it
from kaggle_environments import make
env = make("connectx", debug=True)
env.run(["random", "random"])

render_board(env.state[0].observation, env.configuration)